In [1]:
class A(object):
    def __del__(self): #析构函数
        print('in __del__') #演示过程


In [2]:
a = A()

In [3]:
import sys
sys.getrefcount(a) #查看对象的引用计数

2

In [4]:
sys.getrefcount?
# 比预期多1个(2>1)，是因为调用函数时，其作为参数传递，临时增加了引用数

In [5]:
a2 = a
sys.getrefcount(a) - 1

2

In [6]:
del a2
sys.getrefcount(a) - 1

1

In [7]:
a = 5 #析构函数被调用

in __del__


In [14]:
class Data(object):
    def __init__(self, value, owner):
        self.owner = owner
        self.value = value
    
    def __str__(self):
        return "%s's data, value is %s" % (self.owner, self.value)
    
    def __del__(self):
        print('in Data.__del__')

class Node(object):
    def __init__(self, value):
        self.data = Data(value, self)
    
    def __del__(self):
        print('in Node.__del__')

In [21]:
node = Node(100)
del node
input('wait...') # 可以看出，del node并没有立刻进行析构，可能在未来某一不确定时刻被回收

wait...


''

In [26]:
node = Node(100)
import gc
gc.collect() # 因为定义了__del__析构函数，亦无法使用gc强制回收
             # Python3 中可以强制回收：回收全部已创建的变量
input('wait...')

in Node.__del__
in Data.__del__
wait...


''

In [27]:
a = A()
sys.getrefcount(a) - 1

1

In [28]:
import weakref
a_wref = weakref.ref(a)
a2 = a_wref() #对a进行弱引用
a is a2

True

In [29]:
sys.getrefcount(a) - 1

2

In [30]:
del a

In [31]:
del a2

in __del__


In [32]:
#此时, a_wref()返回None
a_wref() is None #对象存在，返回其引用；不存在，返回None

True

In [33]:
import weakref
class Data(object):
    def __init__(self, value, owner):
        self.owner = weakref.ref(owner) #弱引用具有临时性，尽在使用时产生实际引用，增加计数
        self.value = value
    
    def __str__(self):
        return "%s's data, value is %s" % (self.owner(), self.value)
    
    def __del__(self):
        print('in Data.__del__')

class Node(object):
    def __init__(self, value):
        self.data = Data(value, self)
    
    def __del__(self):
        print('in Node.__del__')


In [34]:
node = Node(100)
del node #使用了弱引用，被正常析构
input('wait...')

in Node.__del__
in Data.__del__
wait...


''